In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Groupby apply on large (relational) data set

## Attentions all writen functions assume a data frame where the date is sorted!!

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,Kazakhstan,0
2,2020-01-22,no,Kenya,0
3,2020-01-22,no,"Korea, South",1
4,2020-01-22,no,Kosovo,0


# Test data

In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]


In [4]:
test_data.head()


,date,state,country,confirmed
15620,2020-03-21,no,Germany,22213
15658,2020-03-21,no,US,25600
15884,2020-03-22,no,Germany,24873
15922,2020-03-22,no,US,33276
16148,2020-03-23,no,Germany,29056


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-04-20,no,147065
US,2020-04-20,no,784326


In [6]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope




In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-04-20     147065
      US      2020-04-20     784326

In [8]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [9]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [10]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15620          NaN
                15884          NaN
                16148     7.417994
                16412     7.142035
                16676     8.012983
                16939     6.954407
                17202     6.501919
                17468     7.390371
                17731    10.136671
                17996    13.541893
                18258    13.781393
                18522    13.140681
                18788    12.037271
                19051    12.735506
                19316    16.052694
                19580    21.372453
                19844    27.427355
                20106    27.511936
                20372    21.792179
                20635    21.495848
                20898    26.565108
                21164    36.198404
                21427    43.982990
                21689    49.423444
                21956    74.043747
                22219    56.424411
                22481    42.468318
                22748    41.52598

In [11]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,463,NaN
2,Alberta,Canada,726,NaN
3,Alberta,Canada,958,NaN
4,Alberta,Canada,1224,NaN


In [13]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,Kazakhstan,0
2,2,2020-01-22,no,Kenya,0
3,3,2020-01-22,no,"Korea, South",1
4,4,2020-01-22,no,Kosovo,0


In [14]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,Kazakhstan,0,NaN
2,2,2020-01-22,no,Kenya,0,NaN
3,3,2020-01-22,no,"Korea, South",1,NaN
4,4,2020-01-22,no,Kosovo,0,NaN


In [15]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply 

In [16]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [17]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [18]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,Kazakhstan,0,NaN,0.0
2,2,2020-01-22,no,Kenya,0,NaN,0.0
3,3,2020-01-22,no,"Korea, South",1,NaN,0.8
4,4,2020-01-22,no,Kosovo,0,NaN,0.0


# Filtered doubling rate

In [19]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
23755,no,Zimbabwe,22533,11.481481
23756,no,Zimbabwe,22799,14.000000
23757,no,Zimbabwe,23001,23.600000
23758,no,Zimbabwe,23296,54.296296
23759,no,Zimbabwe,23759,49.800000


In [20]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
23755,23755,2020-04-20,no,Bahrain,1907,27.666667,1911.2,3.343604e+01
23756,23756,2020-04-20,no,Bangladesh,2948,6.258706,2865.6,7.503264e+00
23757,23757,2020-04-20,no,Barbados,75,inf,75.0,-5.277656e+15
23758,23758,2020-04-20,Zhejiang,China,1268,inf,1268.0,-3.717815e+15
23759,23759,2020-04-20,no,Zimbabwe,25,inf,25.4,4.980000e+01


In [21]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [22]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
22481,22481,2020-04-16,no,Germany,137698,42.468318,137709.8,50.362734
22748,22748,2020-04-17,no,Germany,141397,41.525988,140474.8,50.838457
23011,23011,2020-04-18,no,Germany,143342,49.898063,142937.2,53.706980
23274,23274,2020-04-19,no,Germany,145184,75.684007,145189.3,60.607530
23539,23539,2020-04-20,no,Germany,147065,78.000000,147441.4,64.468407


In [23]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)